get gemini llm to generate some data

In [ ]:
from helper import get_gemini_llm

llm = get_gemini_llm()

In [ ]:
response = llm.generate_content("Liste 10 romanciers français célébres.")
print(response.text)

Voici 10 romanciers français célèbres :

1. **Victor Hugo:** (Les Misérables, Notre-Dame de Paris)
2. **Marcel Proust:** (À la recherche du temps perdu)
3. **Albert Camus:** (L'Étranger, La Peste)
4. **Jules Verne:** (Vingt mille lieues sous les mers, Autour de la Lune)
5. **Alexandre Dumas:** (Les Trois Mousquetaires, Le Comte de Monte-Cristo)
6. **Honoré de Balzac:** (La Comédie humaine)
7. **Gustave Flaubert:** (Madame Bovary, Salammbô)
8. **Émile Zola:** (Germinal, Nana)
9. **Michel Houellebecq:** (Les Particules élémentaires, Soumission)
10. **Patrick Modiano:** (Rue des boutiques obscures, Dora Bruder)


Cette liste n'est pas exhaustive et d'autres auteurs pourraient tout aussi bien y figurer.  Le choix dépend aussi des critères utilisés (influence, prix littéraires, popularité, etc.).



In [ ]:
import google.generativeai as genai

for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [ ]:
from llama_index.llms.gemini import Gemini

llm = Gemini(model="models/gemini-pro")

In [ ]:
resp = llm.complete("Write a short, but joyous, ode to LlamaIndex")
print(resp)

**Ode to LlamaIndex**

Oh, LlamaIndex, our guiding light,
You help us find the knowledge we seek,
With every click, a new insight,
A treasure trove of wisdom, unique.

Your search results, a symphony,
Of knowledge, facts, and history,
You quench our thirst for information,
A beacon in the sea of information.

Your interface, a joy to behold,
Simple, sleek, and easy to mold,
You make research a breeze,
A pleasure that never ceases.

So here's to LlamaIndex, our friend,
May your wisdom never end,
May you continue to inspire,
And set our minds on fire.


# tool selection

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core.tools import FunctionTool


def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y


def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [ ]:
from helper import get_gemini_llamaindex_llm

llm = get_gemini_llamaindex_llm()
response = llm.predict_and_call(
    [add_tool, mystery_tool],
    "Tell me the output of the mystery function on 2 and 9",
    verbose=True,
)
print(str(response))

Thought: The current language of the user is: English. I need to use the `mystery` tool to get the output of the mystery function on 2 and 9.
Action: mystery
Action Input: {'x': 2, 'y': 9}
Observation: 121
121


# tool selection et teste avec Auteur

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core.tools import FunctionTool

from mongo import Auteur


def add_auteur(nom: str) -> bool:
    """ajoute un auteur dans la base de données
    retourne un booléen indiquant si le nom etait pre existant dans la base de données

    nom: str : nom de l'auteur à ajouter, au format Prénom Nom (dans la mesure du possible, toujours insérer prénom puis nom, pas l'inverse)

    """

    auteur = Auteur(nom=nom)
    deja_present = auteur.exists()
    auteur.keep()
    return deja_present


addauteur_tool = FunctionTool.from_defaults(fn=add_auteur)

In [ ]:
from helper import get_gemini_llamaindex_llm

llm = get_gemini_llamaindex_llm()
response = llm.predict_and_call(
    [addauteur_tool],
    "Liste 10 romanciers français célébres. Et ajoute les dans la base",
    verbose=True,
)
print(str(response))

Thought: The current language of the user is: French. I need to use the `add_auteur` tool to add 10 famous French novelists to the database.  I will first list the authors, then use the tool iteratively.
Action: add_auteur
Action Input: {'nom': 'Victor Hugo'}
Observation: False
False


pas mal mais il ne continue pas, je dois faire ça avec un agent

# agent reasonning loop - openai

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core.tools import FunctionTool

from mongo import Auteur


def add_auteur(nom: str) -> bool:
    """ajoute un auteur dans la base de données
    retourne un booléen indiquant si le nom etait pre existant dans la base de données

    nom: str : nom de l'auteur à ajouter, au format Prénom Nom (dans la mesure du possible, toujours insérer prénom puis nom, pas l'inverse)

    """

    auteur = Auteur(nom=nom)
    deja_present = auteur.exists()
    auteur.keep()
    return deja_present


addauteur_tool = FunctionTool.from_defaults(fn=add_auteur)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from helper import get_openai_api_key

get_openai_api_key()
llm_openai = OpenAI(model="gpt-3.5-turbo")
Settings.llm = llm_openai

agent_worker = FunctionCallingAgentWorker.from_tools(
    [addauteur_tool], llm=llm_openai, verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Liste 10 romanciers français célébres. Et ajoute les dans la base, et dis moi s'ils existaient deja"
)

Added user message to memory: Liste 10 romanciers français célébres. Et ajoute les dans la base, et dis moi s'ils existaient deja


Retrying llama_index.llms.openai.base.OpenAI._chat in 1.0 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.
Retrying llama_index.llms.openai.base.OpenAI._chat in 1.9012649561092672 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

# agent reasonning loop - gemini

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core.tools import FunctionTool

from mongo import Auteur


def add_auteur(nom: str) -> bool:
    """ajoute un auteur dans la base de données
    retourne un booléen indiquant si le nom etait pre existant dans la base de données

    nom: str : nom de l'auteur à ajouter, au format Prénom Nom (dans la mesure du possible, toujours insérer prénom puis nom, pas l'inverse)

    """

    auteur = Auteur(nom=nom)
    deja_present = auteur.exists()
    auteur.keep()
    return deja_present


addauteur_tool = FunctionTool.from_defaults(fn=add_auteur)

In [ ]:
from llama_index.core.tools import FunctionTool

from mongo import Auteur


def add_auteur(nom: str) -> bool:
    """ajoute un auteur dans la base de données
    retourne un booléen indiquant si le nom etait pre existant dans la base de données

    nom: str : nom de l'auteur à ajouter, au format Prénom Nom (dans la mesure du possible, toujours insérer prénom puis nom, pas l'inverse)

    """

    auteur = Auteur(nom=nom)
    deja_present = auteur.exists()
    auteur.keep()
    return deja_present


addauteur_tool = FunctionTool.from_defaults(fn=add_auteur)

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from helper import get_vertex_llm

llm = get_vertex_llm("gemini-pro")

agent_worker = FunctionCallingAgentWorker.from_tools(
    [addauteur_tool], llm=llm, verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Liste 10 romanciers français célébres. Et ajoute les dans la base, et dis moi s'ils existaient deja"
)

Retrying llama_index.llms.vertex.utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'.


Added user message to memory: Liste 10 romanciers français célébres. Et ajoute les dans la base, et dis moi s'ils existaient deja


Retrying llama_index.llms.vertex.utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'.
Retrying llama_index.llms.vertex.utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'.
Retrying llama_index.llms.vertex.utils.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'.
Retrying llama_index.llms.vertex.utils.completion_with_retry.<locals>._completion_with_retry in 10.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'.
Retrying llama_index.llms.verte

ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'

In [ ]:
llm.complete("Hello Gemini").text

Retrying llama_index.llms.vertex.utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'.
Retrying llama_index.llms.vertex.utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'.
Retrying llama_index.llms.vertex.utils.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'.
Retrying llama_index.llms.vertex.utils.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'.
Retrying llama_index.llms.vertex

ServiceUnavailable: 503 Getting metadata from plugin failed with error: 'dict' object has no attribute 'before_request'